# Combining and Cleaning Data in Python

In this tutorial, we have a dataset of restaurants in the great Los Angeles area that needs cleaning. In fact, we have two datasets that need to joined, then cleaned. The tasks we need to accomplish are: importing the data to dataframes, combining the dataframes, cleaning up the dataframes (stripping whitespace, removing unwanted periods, remapping categories for 'city' and 'type'), and removing duplicates. I will show you how to check for exact duplicates and near exact duplicates after joining and cleaning the data and then I will show you how to check for duplicates prior to joining the dataframes and then joining them after removing the duplicates. 

In [ ]:
# Importing tutorial packages
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import recordlinkage
import string

# Importing tutorial datasets as DataFrames, replace the ... with your path to the file
restaurants = pd.read_csv('.../restaurants_L2.csv', index_col = 'Unnamed: 0')
restaurants_new = pd.read_csv('.../restaurants_L2_dirty.csv', index_col = 'Unnamed: 0')


In [ ]:
# Looking over the data
restaurants.head()

,name,addr,city,phone,type
0,arnie morton's of chicago,435 s. la cienega blv .,los angeles,3102461501,american
1,art's delicatessen,12224 ventura blvd.,studio city,8187621221,american
2,campanile,624 s. la brea ave.,los angeles,2139381447,american
3,fenix,8358 sunset blvd. west,hollywood,2138486677,american
4,grill on the alley,9560 dayton way,los angeles,3102760615,american


In [ ]:
restaurants_new.head()

,name,addr,city,phone,type
0,kokomo,6333 w. third st.,la,2139330773,american
1,feenix,8358 sunset blvd. west,hollywood,2138486677,american
2,parkway,510 s. arroyo pkwy .,pasadena,8187951001,californian
3,r-23,923 e. third st.,los angeles,2136877178,japanese
4,gumbo,6333 w. third st.,la,2139330358,cajun/creole


In [ ]:
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 0 to 335
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    336 non-null    object
 1   addr    336 non-null    object
 2   city    336 non-null    object
 3   phone   336 non-null    int64 
 4   type    336 non-null    object
dtypes: int64(1), object(4)
memory usage: 15.8+ KB


In [ ]:
restaurants_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 81
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    82 non-null     object
 1   addr    82 non-null     object
 2   city    82 non-null     object
 3   phone   82 non-null     int64 
 4   type    82 non-null     object
dtypes: int64(1), object(4)
memory usage: 3.8+ KB


## Dataset Summaries

The original 'restaurants' data contains 336 rows of data. The variables include 'name', 'addr' (address), 'city', 'phone', and 'type'. The data that needs to be joined from 'restaurants_new' has 82 rows and the same variables. At present, we will strip whitespace; title case 'name', 'address', and 'type'; check for consistencies in 'city', 'phone', and 'type'; change 'type' to a categorical variable; reformat the 'phone' variable, and finally, check for duplicates. You can find the datasets on my Github. Let's get to it!

## Joining Dataframes

I will not get into all the specifics about joining data and I utilize the word join lightly, when in reality we are just merging two dataframes. Joins, if you are familar with SQL, take on numerous forms. The below code is going to combine our two datasets, but it will mess up the row index, so we will have to reset that as well.

For more on Pandas joins, see this link <https://realpython.com/pandas-merge-join-and-concat/>. 

In [ ]:
# Join the Data
df = pd.concat([restaurants, restaurants_new], axis=0)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    418 non-null    object
 1   addr    418 non-null    object
 2   city    418 non-null    object
 3   phone   418 non-null    int64 
 4   type    418 non-null    object
dtypes: int64(1), object(4)
memory usage: 16.5+ KB


Now that we have merged/joined/concatenated the two datasets, let's strip some whitespace.

In [ ]:
cols = df.select_dtypes('object').columns
df[cols] = df[cols].apply(lambda x: x.str.strip())
df.head()

,name,addr,city,phone,type
0,arnie morton's of chicago,435 s. la cienega blv .,los angeles,3102461501,american
1,art's delicatessen,12224 ventura blvd.,studio city,8187621221,american
2,campanile,624 s. la brea ave.,los angeles,2139381447,american
3,fenix,8358 sunset blvd. west,hollywood,2138486677,american
4,grill on the alley,9560 dayton way,los angeles,3102760615,american


Hopefully all the whitespace is gone. Let's move on to changing everything to title case. We can utilize the 'cols' we created above to make this as simple as possible. I have read numerous articles indicating that using lamda in code equates to poor coding, but as you can see in the above and below example, it makes certain tasks extremely easy. One issue that I ran into my original code (commented out) was anything after an apostrophe was capitalized with .title(). Truthfully, I had to reach out for help on this issue and received two ingenious ways to fix it.

In [ ]:
# Most efficient way is to prevent the problem.
# import string
# Three different basic Python classes and not one covered this ingenious method
df[cols] = df[cols].applymap(string.capwords)

# Original Way I started with a lambda
# df[cols] = df[cols].apply(lambda x: x.str.title())
# The following line was the fix
#df['name'] = df['name'].replace("\'S", "\'s", regex=True)

df.head()

,name,addr,city,phone,type
0,Arnie Morton's Of Chicago,435 S. La Cienega Blv .,Los Angeles,3102461501,American
1,Art's Delicatessen,12224 Ventura Blvd.,Studio City,8187621221,American
2,Campanile,624 S. La Brea Ave.,Los Angeles,2139381447,American
3,Fenix,8358 Sunset Blvd. West,Hollywood,2138486677,American
4,Grill On The Alley,9560 Dayton Way,Los Angeles,3102760615,American


In [ ]:
# Let's get rid of periods
df[cols] = df[cols].apply(lambda x: x.str.replace('.' or ' .', ''))
df[cols] = df[cols].apply(lambda x: x.str.strip())
df.head()

,name,addr,city,phone,type
0,Arnie Morton's Of Chicago,435 S La Cienega Blv,Los Angeles,3102461501,American
1,Art's Delicatessen,12224 Ventura Blvd,Studio City,8187621221,American
2,Campanile,624 S La Brea Ave,Los Angeles,2139381447,American
3,Fenix,8358 Sunset Blvd West,Hollywood,2138486677,American
4,Grill On The Alley,9560 Dayton Way,Los Angeles,3102760615,American


We could go in and fix all the instances with Boulevard is abbreviated with Blv instead of Blvd, S for south, and so on. But I will not go over that here. You can utilize the .replace() like I did with periods above. The next problem to tackle is to check if any values in 'city' and 'type' need to be condensed. This will be a multistep process for each column.

Step 1: Get the unique values. I like to do it with `.value_counts()`.
Step 2: Remap incorrect values to corrected values.
Step 3: Recheck the unique values.

In [ ]:
# Step 1: 'city'
df.city.value_counts()

New York             182
San Francisco         55
Los Angeles           51
Atlanta               33
Las Vegas             20
La                    13
Santa Monica          12
Beverly Hills          7
Pasadena               4
West La                4
Studio City            3
Hollywood              3
Venice                 3
Encino                 2
W Hollywood            2
Westwood               2
Westlake Village       2
Northridge             1
Redondo Beach          1
Chinatown              1
Mar Vista              1
Rancho Park            1
Marina Del Rey         1
Toluca Lake            1
Seal Beach             1
Long Beach             1
Culver City            1
Pacific Palisades      1
St Hermosa Beach       1
Los Feliz              1
Malibu                 1
Century City           1
Monterey Park          1
St Boyle Hts           1
Burbank                1
Brooklyn               1
Sherman Oaks           1
Name: city, dtype: int64

La needs to be remapped to Los Angeles. I will also map West La to West Los Angeles and W Hollywood to West Hollywood.

In [ ]:
df.city = df.city.replace(to_replace=['La'], value='Los Angeles')
df.city = df.city.replace(to_replace=['West La'], value='West Los Angeles')
df.city = df.city.replace(to_replace=['W Hollywood'], value='West Hollywood')
df.city.value_counts()

New York             182
Los Angeles           64
San Francisco         55
Atlanta               33
Las Vegas             20
Santa Monica          12
Beverly Hills          7
West Los Angeles       4
Pasadena               4
Venice                 3
Studio City            3
Hollywood              3
Westwood               2
Encino                 2
Westlake Village       2
West Hollywood         2
Toluca Lake            1
Northridge             1
Chinatown              1
Mar Vista              1
Rancho Park            1
Marina Del Rey         1
Redondo Beach          1
Culver City            1
Long Beach             1
Pacific Palisades      1
St Hermosa Beach       1
Los Feliz              1
Malibu                 1
Century City           1
Monterey Park          1
St Boyle Hts           1
Burbank                1
Brooklyn               1
Seal Beach             1
Sherman Oaks           1
Name: city, dtype: int64

In [ ]:
df.type.value_counts()

American            157
Italian              85
Asian                76
Coffeebar            26
Mexican              12
Southern              8
Steakhouses           6
Californian           5
Coffee Shops          5
Diners                3
Hamburgers            3
Hot Dogs              3
Delis                 3
Seafood               2
Noodle Shops          2
Pacific New Wave      2
Cajun/creole          1
Chicken               1
Continental           1
Russian               1
Chinese               1
French ( New )        1
American ( New )      1
Fast Food             1
Japanese              1
Desserts              1
Southern/soul         1
Mexican/tex-mex       1
Cajun                 1
Health Food           1
Pizza                 1
Indonesian            1
Eclectic              1
Vietnamese            1
Southwestern          1
Middle Eastern        1
Name: type, dtype: int64

In [ ]:
# A little more work to be done
df.type = df.type.replace(to_replace=['American ( New )', 'Hot Dogs', 'Hamburgers', 'Diners', 'Chicken', 'Continental'], value='American')
df.type = df.type.replace(to_replace=['Indonesian', 'Japanese', 'Vietnamese', 'Chinese', 'Noodle Shops', 'Pacific New Wave'], value='Asian')
df.type = df.type.replace(to_replace=['Cajun/creole'], value='Cajun')
df.type = df.type.replace(to_replace=['Mexican/tex-mex', 'Southwestern'], value='Mexican')
df.type = df.type.replace(to_replace=['Southern/soul'], value='Southern')
df.type = df.type.replace(to_replace=['Coffeebar'], value='Coffee Shops')
df.type.value_counts()

American          169
Italian            85
Asian              84
Coffee Shops       31
Mexican            14
Southern            9
Steakhouses         6
Californian         5
Delis               3
Seafood             2
Cajun               2
Desserts            1
Fast Food           1
Eclectic            1
Health Food         1
Pizza               1
Middle Eastern      1
French ( New )      1
Russian             1
Name: type, dtype: int64

Now that we have a little more of a managable list in both 'city' and 'type', let's get to checking for duplicates. We will first check for exact matches. I do not expect there to be any, but let's try. Then we will check for matches based upon a couple columns. I'm going to utilized 'city' and 'type' since we just cleaned those up and the 'phone' column. I will then sort the results (if any) by phone.

In [ ]:
# Let's check for exact duplicates first
duplicates = df.duplicated()
df[duplicates]
# Looks Empty

,name,addr,city,phone,type


In [ ]:
# Let's check for partial duplicates
columns_check = ['city', 'phone', 'type']
dup_2 = df.duplicated(subset = columns_check, keep = False)
df[dup_2].sort_values(by='phone')

,name,addr,city,phone,type
14,Aureole,34 E 61st St,New York,2123191660,American
403,Aureola,34 E 61st St,New York,2123191660,American
18,Dawat,210 E 58th St,New York,2123557555,Asian
407,Dawatt,210 E 58th St,New York,2123557555,Asian
16,Carmine's,2450 Broadway Between 90th And 91st Sts,New York,2123622200,Italian
393,Carrmine's,2450 Broadway Between 90th And 91st Sts,New York,2123622200,Italian
15,Cafe Lalo,201 W 83rd St,New York,2124966031,Coffee Shops
391,Cafe Lala,201 W 83rd St,New York,2124966031,Coffee Shops
343,21 Clubs,21 W 52nd St,New York,2125827200,American
13,21 Club,21 W 52nd St,New York,2125827200,American


We have at least 22 duplicates. My plan, although this would be time consuming and subject error in large datasets, is to drop by a chosen index. We will then have to reset the index. We will remove index ['403', '18', '393', '15', '343', '363', '356', '19', '348', '411', '386', '5', '337', '410', '376', '6', '389', '8', '415', '357', '362', '364']. I'm noticing a lot of high index numbers, I think the data set we joined had most of the errors. In a real scenario, we may want to reconsider that data source in the future.

In [ ]:
df.drop(df.index[[403,18,393,15,343,363,356,19,348,411,386,5,337,410,376,6,389,8,415,357,362,364]], inplace=True)

In [ ]:
df.info()
dup_3 = df.duplicated(subset = columns_check, keep = False)
df[dup_3].sort_values(by='phone')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396 entries, 0 to 417
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    396 non-null    object
 1   addr    396 non-null    object
 2   city    396 non-null    object
 3   phone   396 non-null    int64 
 4   type    396 non-null    object
dtypes: int64(1), object(4)
memory usage: 18.6+ KB


,name,addr,city,phone,type


In [ ]:
df.reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396 entries, 0 to 417
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    396 non-null    object
 1   addr    396 non-null    object
 2   city    396 non-null    object
 3   phone   396 non-null    int64 
 4   type    396 non-null    object
dtypes: int64(1), object(4)
memory usage: 34.7+ KB


## Example Two

The next few lines of code will merge the data frames after comparing and potentially eliminating possible matches. This will require cleaning as done above after the merge. In the code below, you will see 'indexer.block('city')'. This functions like an SQL join where you specify the primary key and foreign key. In this instance, there are no primary keys or foreign keys, so you can select the column that likely has the best matches. Truthfully, 'phone' would be a better choice as we saw when searching for duplicates in the previous example. Further down in the code, you will see 'compare_cl.exact()' and 'compare_cl.string()'. When utilizing .exact() you are telling the comparison it wants, well exact matches. This works well with integers, decimals (with rounding) gets a little tricky. The .string() you specify a threshold. This tells the comparison what percentage of the string should match. I selected 80%. The lower the threshold, the more matches. The output will look similar to a correlation matrix; however, for each comparison column you specify, the result will be a 0 or 1. To be considered a duplicate, you obviously want more 1s. 

In [ ]:
# Let's check it with recordlinkage
# import recordlinkage (if not already done so)

indexer = recordlinkage.Index()
indexer.block('city')
full_pairs = indexer.index(restaurants, restaurants_new)

#Comparison
compare_cl = recordlinkage.Compare()
compare_cl.exact('city', 'city', label='city')
compare_cl.string('name', 'name', threshold=0.8, label='name')
compare_cl.string('addr', 'addr', threshold=0.8, label='addr')

potential_matches = compare_cl.compute(full_pairs, restaurants, restaurants_new)

potential_matches

city  name  addr
0   3      1   0.0   0.0
    26     1   0.0   0.0
    40     1   1.0   1.0
    43     1   0.0   0.0
    50     1   0.0   0.0
...      ...   ...   ...
247 47     1   0.0   0.0
    55     1   0.0   0.0
    57     1   0.0   0.0
    67     1   0.0   0.0
    71     1   0.0   0.0

[2047 rows x 3 columns]

We have a lot of potential matches. Let's narrow those down. As indicated above, the more 1s, the more likely it is a duplicate. So let's find the potential duplicates that have a 1 in all three comparison columns. 

In [ ]:
matches = potential_matches[potential_matches.sum(axis = 1) == 3]
matches

,,city,name,addr
0,40,1,1.0,1.0
2,74,1,1.0,1.0
4,53,1,1.0,1.0
5,65,1,1.0,1.0
6,73,1,1.0,1.0
8,43,1,1.0,1.0
9,50,1,1.0,1.0
10,75,1,1.0,1.0
12,26,1,1.0,1.0
1,28,1,1.0,1.0


At this point, we can see that we have 22 duplicates. If you want to continue to merge without the duplicates then clean as we did above, the code for the merge is below. I will not go through the cleaning again. You will need to reset the index after the merge.

In [ ]:
duplicate_rows = matches.index.get_level_values(1)
rest_new_dups = restaurants_new[restaurants_new.index.isin(duplicate_rows)]
rest_new_new = restaurants_new[~restaurants_new.index.isin(duplicate_rows)]
df_new = restaurants.append(rest_new_new)
df_new.reset_index()
df_new.info() # We should have 396 rows of data.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396 entries, 0 to 81
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    396 non-null    object
 1   addr    396 non-null    object
 2   city    396 non-null    object
 3   phone   396 non-null    int64 
 4   type    396 non-null    object
dtypes: int64(1), object(4)
memory usage: 18.6+ KB


In [ ]:
# Let's admire our work from the first example
df.head()

## Conclusion
This tutorial covered joining data then removing duplicates, removing duplicates then joining data, cleaning up some categories and changing strings to title case. This also completes my little series on cleaning data in Pandas. If you have not seen the other three, please check out either my GitHub or Medium page. As always, I do receive any monetary value for sharing writing and sharing these tutorials on Medium, nor will I ever. I do this to solidify what I have learned and hopefully help a few people out along the way. As a reminder, I feel cleaning data is the single most important step to great analysis, as the old saying goes "garbage in, garbage out". Take your time, make sure that you are thorough. While you are cleaning, you will become more familar with the data and potentially help to fix your organization's data problems as you do it.